LightGBM : 91.5
XGboost :
ExtraTreesRegressor : 90
RandomForestRegressor : 


In [1]:
import pandas as pd
import pickle

In [2]:
# Importation des données et visualisation 
df = pd.read_csv('datasets/velille.csv', parse_dates=True, index_col='datetime') 
df

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,day,hour,year,date,is_night
datetime,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,Winter,0,0,Clear,9.84,14.395,81,0.0000,3,13,16,1,Saturday,0,2011,718,1
2011-01-01 01:00:00,Winter,0,0,Clear,9.02,13.635,80,0.0000,8,32,40,1,Saturday,1,2011,718,1
2011-01-01 02:00:00,Winter,0,0,Clear,9.02,13.635,80,0.0000,5,27,32,1,Saturday,2,2011,718,1
2011-01-01 03:00:00,Winter,0,0,Clear,9.84,14.395,75,0.0000,3,10,13,1,Saturday,3,2011,718,1
2011-01-01 04:00:00,Winter,0,0,Clear,9.84,14.395,75,0.0000,0,1,1,1,Saturday,4,2011,718,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-19 19:00:00,Fall,0,1,Clear,15.58,19.695,50,26.0027,7,329,336,12,Wednesday,19,2012,0,0
2012-12-19 20:00:00,Fall,0,1,Clear,14.76,17.425,57,15.0013,10,231,241,12,Wednesday,20,2012,0,1
2012-12-19 21:00:00,Fall,0,1,Clear,13.94,15.910,61,15.0013,4,164,168,12,Wednesday,21,2012,0,1


In [3]:
# Remise en nombre du temps et des saisons par ordre

weather_dict = {'Clear':1, 'Misty+Cloudy':2, 'Light Snow/Rain':3, 'Heavy Snow/Rain':4}
season_dict = {'Winter':1, 'Fall':2, 'Spring':3, 'Summer':4}
df['weather'] = df['weather'].map(weather_dict)
df['season'] = df['season'].map(season_dict)

In [4]:
# Séparations données de test

train_df = df[df.index <= '2012-08-31'].drop(['casual','registered'], axis=1)
test_df = df[df.index >= '2012-09-01'].drop(['casual','registered'], axis=1)



In [5]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()

In [6]:
train_df = train_df.drop(['datetime'], axis=1)
test_df = test_df.drop(['datetime'], axis=1)

y_train = train_df['count']
X_train = train_df.drop(['count'], axis=1)
y_test = test_df['count']
X_test = test_df.drop(['count'], axis=1)

In [50]:
# Import XBBRegressor

xgb = pickle.load(open('XGBRegressor.pkl', 'rb'))
result = xgb.score(X_test, y_test)
print(result)

0.9058640431829525


In [8]:
# Import XGB Florian

# xgb_flo = pickle.load(open('xgb_best.pkl', 'rb'))
# result = xgb_flo.score(X_test, y_test)
# print(result)

In [51]:
# Import LGBMRegressor

lgbm = pickle.load(open('LGBMRegressor.pkl', 'rb'))
result = lgbm.score(X_test, y_test)
print(result)

0.9210319440325717


In [52]:
# Import RandomForestRegressor

rf = pickle.load(open('RandomForestRegressor.pkl', 'rb'))
result = rf.score(X_test, y_test)
print(result)

0.8813124774734877


In [66]:
# Import ExtraTreesRegressor

extra = pickle.load(open('ExtraTreesRegressor2.pkl', 'rb'))
result = extra.score(X_test, y_test)
print(result)

0.9009600585154496


In [113]:
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

base_models = [('LightGBM', lgbm),
               ]
meta_model = LinearRegression()
stacking_model = StackingRegressor(estimators = base_models, final_estimator = meta_model, cv=2)

In [13]:
# stacking_model.get_params().keys()

In [114]:


stacking_model.fit(X_train, y_train)



StackingRegressor(cv=2,
                  estimators=[('LightGBM',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('pipeline-1',
                                                                                 Pipeline(steps=[('onehotencoder',
                                                                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                                                                 ['season',
                                                                                  'weather',
                                                                                  'holiday',
                                                                                  'workingday',
                                                                                  'year',
                                                 

In [115]:
stacking_model.score(X_test, y_test)

0.9284686246936358

In [85]:
from sklearn.metrics import *

y_pred = stacking_model.predict(X_test)
print('r2', r2_score(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

r2 0.9284686246936358
mae 39.574517844168454


In [87]:
# pickle
import pickle
pickle.dump(stacking_model, open('StackingModel.pkl', 'wb'))

In [88]:
# Test Ridge

from sklearn.linear_model import Ridge
stacking_ridge = StackingRegressor(estimators = base_models, final_estimator = Ridge(), cv=2)

In [89]:
stacking_ridge.fit(X_train, y_train)

StackingRegressor(cv=2,
                  estimators=[('LightGBM',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('pipeline-1',
                                                                                 Pipeline(steps=[('onehotencoder',
                                                                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                                                                 ['season',
                                                                                  'weather',
                                                                                  'holiday',
                                                                                  'workingday',
                                                                                  'year',
                                                 

In [90]:
stacking_ridge.score(X_test, y_test)

# Evaluation du training set

y_train_predict = stacking_ridge.predict(X_train)
r2 = r2_score(y_train, y_train_predict)

print("La performance du modèle sur la base d'apprentissage")
print('le score R2 est {}'.format(r2))
print('mae', mean_absolute_error(y_train, y_train_predict))
print('\n')

print("La performance du modèle sur la base du test")
y_pred = stacking_ridge.predict(X_test)
print('r2', r2_score(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

La performance du modèle sur la base d'apprentissage
le score R2 est 0.9690355524053441
mae 20.27599721882929


La performance du modèle sur la base du test
r2 0.9284686247886903
mae 39.574517783179736


In [45]:
# pickle
import pickle
pickle.dump(stacking_ridge, open('StackingRidge.pkl', 'wb'))

In [91]:
# Test Lasso

from sklearn.linear_model import Lasso
stacking_lasso = StackingRegressor(estimators = base_models, final_estimator = Lasso(), cv=2)

In [92]:
stacking_lasso.fit(X_train, y_train)

StackingRegressor(cv=2,
                  estimators=[('LightGBM',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('pipeline-1',
                                                                                 Pipeline(steps=[('onehotencoder',
                                                                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                                                                 ['season',
                                                                                  'weather',
                                                                                  'holiday',
                                                                                  'workingday',
                                                                                  'year',
                                                 

In [93]:
stacking_lasso.score(X_test, y_test)

0.9284694154024109

In [95]:
# pickle
import pickle
pickle.dump(stacking_lasso, open('StakingLasso3.pkl', 'wb'))

In [94]:

y_pred = stacking_lasso.predict(X_test)
print('r2', r2_score(y_test, y_pred))
print('mae', mean_absolute_error(y_test, y_pred))

r2 0.9284694154024109
mae 39.57400892540746


In [25]:
from sklearn.ensemble import VotingRegressor
models = [('LightGBM', lgbm),
            ('XBG', xgb),
            ('ExtraTrees', extra),
            ('RandomForest', rf),
               ]

voting_model = VotingRegressor(estimators = models)

In [26]:
voting_model.fit(X_train, y_train)

VotingRegressor(estimators=[('LightGBM',
                             Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['season',
                                                                                'weather',
                                                                                'holiday',
                                                                                'workingday',
                                                                                'year',
                                                                                'month',
    

In [27]:
voting_model.score(X_test, y_test)

0.9130906573628933